In [ ]:
#FIRST! Make sure you have a GPU instance colab. Then you can run all cells
!pip install transformers
!pip install torchmetrics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 61.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 34.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 107.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 61.7 MB/s eta 0:00:00


In [ ]:


import torch
import torch.nn.functional as F
from typing import List
from transformers import  AutoTokenizer, AutoModel

from torchmetrics.functional import pairwise_cosine_similarity

def mean_pooling( model_output, attention_mask ):
    """Applies mean pooling to the token embeddings generated by the model.
    Args:
        model_output (torch.Tensor): Embedding model output, where the first element contains token embeddings.
        attention_mask (torch.Tensor): Attention mask to indicate valid tokens.
    Returns:
        torch.Tensor: Mean-pooled representation of the token embeddings.
    Notes:
        - The function calculates the mean-pooled representation using the attention mask for valid tokens.
        - Input_mask_expanded is created by expanding the attention mask to match the size of token embeddings.
        - The result is obtained by summing the element-wise multiplication of embeddings and input_mask_expanded,
            and dividing it by the sum of input_mask_expanded after clamping its values to a minimum of 1e-9.
    """
    token_embeddings = model_output[0]
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(
        input_mask_expanded.sum(1), min=1e-9
    )

class DiversityRewardModel(  ):


    def __init__( self, device: str ):
        super().__init__()
        self.device = device
        diversity_model_path = "sentence-transformers/all-mpnet-base-v2"
        self.tokenizer = AutoTokenizer.from_pretrained( diversity_model_path )
        self.model = AutoModel.from_pretrained( diversity_model_path ).to(self.device)
        self.reward_quantile = torch.tensor(0.1).to(self.device)

    def get_embeddings( self, sentences: List[str] ) -> "torch.FloatTensor":
        """Runs a forward pass through the model.
        Args:
            sentences (:obj:`List[str]`):
                text message to be encoded.
        Returns:
            embedding (:obj:`torch.FloatTensor`):
                Embedding for the message.
        """
        # Tokenizing sentences
        encoded_input = self.tokenizer(
            sentences,
            padding=True,
            truncation=True,
            return_tensors="pt",
        ).to(self.device)

        # Compute token embedding
        with torch.no_grad():
            embeddings = self.model(**encoded_input)

        # Pooling
        sentence_embeddings = mean_pooling(embeddings, encoded_input["attention_mask"])

        # Normalizing
        sentence_embeddings = F.normalize(sentence_embeddings, p=2, dim=1)
        return sentence_embeddings

    def get_rewards( self, completions ) :

        # Get embeddings for all completions.
        embeddings = self.get_embeddings( completions )

        # Calculate the pairwise cosine similarity.
        similarity = pairwise_cosine_similarity( embeddings, embeddings )

        return similarity.tolist()
        # Reward to be at the 10% quantile of the 1 - similarity score.
        rewards = (1 - similarity).quantile(self.reward_quantile, dim = 1 )

        # Return all
        return rewards




cl = DiversityRewardModel("cuda")

In [ ]:
responses = ["Bonjour, je m'appelle Viktor.","Hello, my name is Viktor"]

similarity = cl.get_rewards(responses)
print(similarity)

[[1.0000001192092896, 0.5952279567718506], [0.5952279567718506, 1.0000001192092896]]


In [ ]:
def regularise( rewards ):
    # sigmoid function that maps 0.07 -> 0.23; 0.1 -> 0.5; 0.2 -> 0.98
    return 1/(1 + torch.exp(-40 * rewards + 4))


print(regularise(torch.tensor([1 - 0.9396640062332153])))

tensor([0.1699])


In [ ]:
reward = (open_assitant_score *0.6 + reciprocate_score *0.4) * relevance * diversity